# Clasificación temática - Bosque/no bosque
Implementación de generación del mapa de clasificación temática utilizando las consultas a bajo nivel que ofrece el cubo. 
## Parámetros

In [16]:
%matplotlib inline
longs=(-75, -74)
lats=(5, 6)
normalized=True
time_range=("2000-01-01","2000-12-31")
bands=["blue","green","red","nir", "swir1","swir2"]
pdct='ls7_ledaps_utm'
vegetation_rate = 0.7
ndvi_threshold = 0.3
slice_size = 3
ok_pixels = 1

## CONSULTA

In [17]:
import datacube
import numpy as np
dc = datacube.Datacube(app='dc-prueba2')
minValid=1;
nbar = dc.load(product=pdct, longitude=longs, latitude=lats, time=time_range)

## Medianas

In [18]:
nodata=-9999
medians={}
for band in bands:
    
    datos=np.where(nbar.data_vars[band]!=-9999,nbar.data_vars[band], np.nan)
    allNan=~np.isnan(datos)
    if normalized:
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    medians[band]=np.nanmedian(datos,0)
    medians[band][np.sum(allNan,0)<minValid]=np.nan
del datos    

## NDVI

In [19]:
period_red = medians["red"]
period_nir = medians["nir"]
del medians
mask_nan=np.logical_or(np.isnan(period_red), np.isnan(period_nir))
period_nvdi = np.true_divide( np.subtract(period_nir,period_red) , np.add(period_nir,period_red) )
period_nvdi[mask_nan]=np.nan

## Bosque/no bosque

In [20]:
height = period_nvdi.shape[0]
width = period_nvdi.shape[1]
bosque_nobosque=np.full(period_nvdi.shape, -1)
for y1 in xrange(0, height, slice_size):
    for x1 in xrange(0, width, slice_size):
        x2 = x1 + slice_size
        y2 = y1 + slice_size
        if(x2 > width):
            x2 = width
        if(y2 > height):
            y2 = height
        submatrix = period_nvdi[y1:y2,x1:x2]
        ok_pixels = np.count_nonzero(~np.isnan(submatrix))
        if ok_pixels==0:
            bosque_nobosque[y1:y2,x1:x2] = nodata    
        elif float(np.nansum(submatrix>ndvi_threshold))/float(ok_pixels) >= vegetation_rate :
            bosque_nobosque[y1:y2,x1:x2] = 1
        else:
            bosque_nobosque[y1:y2,x1:x2] = 0

## Guardar a archivo NetCDF el NDVI 

In [21]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
nco=netcdf_writer.create_netcdf("ndvi_bnb_2000.nc")
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, nbar.coords[x].values, nbar.coords[x].units)
    else:
        netcdf_writer.create_coordinate(nco, x, np.array(["t1-t2"]), "period")
netcdf_writer.create_grid_mapping_variable(nco, nbar.crs)
period_nvdi[np.isnan(period_nvdi)]=nodata
var= netcdf_writer.create_variable(nco, 'ndvi', Variable(np.dtype(np.float32), nodata, ('time','latitude', 'longitude'), None) ,set_crs=True)
var[:] = netcdf_writer.netcdfy_data(period_nvdi)
nco.close()

## Guardar a NetCDF el mapa temático

In [22]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
nco=netcdf_writer.create_netcdf("bnb_2000.nc")
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, nbar.coords[x].values, nbar.coords[x].units)
    else:
        netcdf_writer.create_coordinate(nco, x, np.array(["t1-t2"]), "period")
netcdf_writer.create_grid_mapping_variable(nco, nbar.crs)
var= netcdf_writer.create_variable(nco, 'bosque-noBosque', Variable(np.dtype(np.int16), nodata, ('time','latitude', 'longitude'), None) ,set_crs=True)
var[:] = netcdf_writer.netcdfy_data(bosque_nobosque)
nco.close()